# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Perotti  ->  G. Perotti  |  ['G. Perotti']


Zhang  ->  X. Zhang  |  ['Zhang']
R. Launhardt  ->  R. Launhardt  |  ['R. Launhardt']
F. Davies  ->  F. Davies  |  ['F. Davies']
Arxiv has 83 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.13598


extracting tarball to tmp_2308.13598...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure figures/AppendixB_tau.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:692: LatexWarning: Could not find graphic \includegraphics[trim={0 0 0 0}, clip, width=\hsize]{figures/AppendixB_tau.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


...

✔ → 39637:\section{Inference Method}
  ↳ 63033:\section{Low-$z$ \lya  Forests and Simulated Absorbers}


✔ → 63033:\section{Low-$z$ \lya  Forests and Simulated Absorbers}
  ↳ 91422:\section{Summary and Discussion}


✔ → 91422:\section{Summary and Discussion}
  ↳ 101452:\section{ Inference based on the Photoheating labels [$A$,$B$]}


✔ → 101452:\section{ Inference based on the Photoheating labels [$A$,$B$]}
  ↳ 105320:end


/tmp/ipykernel_2170/3009462391.py:49: LatexWarning: 2308.14738 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.13598-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.13598) | **Linking ice and gas in the Coronet cluster in Corona Australis**  |
|| <mark>G. Perotti</mark>, et al. |
|*Appeared on*| *2023-08-29*|
|*Comments*| *20 pages, 17 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| During the journey from the cloud to the disc, the chemical composition of the protostellar envelope material can be either preserved or processed to varying degrees depending on the surrounding physical environment. This works aims to constrain the interplay of solid (ice) and gaseous methanol (CH$_3$OH) in the outer regions of protostellar envelopes located in the Coronet cluster in Corona Australis (CrA), and assess the importance of irradiation by the Herbig Ae/Be star R CrA. CH$_3$OH is a prime test-case as it predominantly forms as a consequence of the solid-gas interplay (hydrogenation of condensed CO molecules onto the grain surfaces) and it plays an important role in future complex molecular processing. We present 1.3 mm Submillimeter Array (SMA) and Atacama Pathfinder Experiment (APEX) observations towards the envelopes of four low-mass protostars in the Coronet. Eighteen molecular transitions of seven species are identified. We calculate CH$_3$OH gas-to-ice ratios in this strongly irradiated cluster and compare them with ratios determined towards protostars located in less irradiated regions such as the Serpens SVS 4 cluster in Serpens Main and the Barnard 35A cloud in the $\lambda$ Orionis region. The CH$_3$OH gas-to-ice ratios in the Coronet vary by one order of magnitude (from 1.2$\times$10$^{-4}$ to 3.1$\times$10$^{-3}$) which is similar to less irradiated regions as found in previous studies. We find that the CH$_3$OH gas-to-ice ratios estimated in these three regions are remarkably similar despite the different UV radiation field intensities and formation histories. This result suggests that the overall CH$_3$OH chemistry in the outer regions of low-mass envelopes is relatively independent of variations in the physical conditions and hence that it is set during the prestellar stage. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.14590-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.14590) | **Twenty-Five Years of Accretion onto the Classical T Tauri Star TW Hya**  |
|| G. J. Herczeg, et al. -- incl., <mark>R. Launhardt</mark> |
|*Appeared on*| *2023-08-29*|
|*Comments*| *Accepted by ApJ. 31 pages*|
|**Abstract**| Accretion plays a central role in the physics that governs the evolution and dispersal of protoplanetary disks. The primary goal of this paper is to analyze the stability over time of the mass accretion rate onto TW Hya, the nearest accreting solar-mass young star. We measure veiling across the optical spectrum in 1169 archival high-resolution spectra of TW Hya, obtained from 1998--2022. The veiling is then converted to accretion rate using 26 flux-calibrated spectra that cover the Balmer jump. The accretion rate measured from the excess continuum has an average of $2.51\times10^{-9}$~M$_\odot$~yr$^{-1}$ and a Gaussian distribution with a FWHM of 0.22 dex. This accretion rate may be underestimated by a factor of up to 1.5 because of uncertainty in the bolometric correction and another factor of 1.7 because of excluding the fraction of accretion energy that escapes in lines, especially Ly$\alpha$. The accretion luminosities are well correlated with He line luminosities but poorly correlated with H$\alpha$ and H$\beta$ luminosity. The accretion rate is always flickering over hours but on longer timescales has been stable over 25 years. This level of variability is consistent with previous measurements for most, but not all, accreting young stars. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.14065-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.14065) | **HybPSF: Hybrid PSF reconstruction for the observed JWST NIRCam image**  |
|| Huanyuan, et al. -- incl., <mark>Zhang</mark> |
|*Appeared on*| *2023-08-29*|
|*Comments*| **|
|**Abstract**| The James Webb Space Telescope (JWST) ushers in a new era of astronomical observation and discovery, offering unprecedented precision in a variety of measurements such as photometry, astrometry, morphology, and shear measurement. Accurate point spread function (PSF) models are crucial for many of these measurements. In this paper, we introduce a hybrid PSF construction method called HybPSF for JWST NIRCam imaging data. HybPSF combines the WebbPSF software, which simulates the PSF for JWST, with observed data to produce more accurate and reliable PSF models. We apply this method to the SMACS J0723 imaging data and construct supplementary structures from residuals obtained by subtracting the WebbPSF PSF model from the data. Our results show that HybPSF significantly reduces discrepancies between the PSF model and the data compared to WebbPSF. Specifically, the PSF shape parameter ellipticity and size comparisons indicate that HybPSF improves precision by a factor of approximately 10 for \$R^2\$ and \$50\%\$ for \$e\$. This improvement has important implications for astronomical measurements using JWST NIRCam imaging data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.14738-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.14738) | **The Impact of the WHIM on the IGM Thermal State Determined from the  Low-$z$ Lyman-$α$ Forest**  |
|| T. Hu, et al. -- incl., <mark>F. Davies</mark> |
|*Appeared on*| *2023-08-29*|
|*Comments*| *23 pages, 24 figures*|
|**Abstract**| At $z \lesssim 1$, shock heating caused by large-scale velocity flows and possibly violent feedback from galaxy formation, converts a significant fraction of the cool gas ($T\sim 10^4$ K) in the intergalactic medium (IGM) into warm-hot phase (WHIM) with $T >10^5$K, resulting in a significant deviation from the previously tight power-law IGM temperature-density relationship, $T=T_0 (\rho / {\bar{\rho}})^{\gamma -1}$. This study explores the impact of the WHIM on measurements of the low-$z$ IGM thermal state, $[T_0,\gamma]$, based on the $b$-$N_{H I}$ distribution of the Lyman-$\alpha$ forest. Exploiting a machine learning-enabled simulation-based inference method trained on Nyx hydrodynamical simulations, we demonstrate that [$T_0$, $\gamma$] can still be reliably measured from the $b$-$N_{H I}$ distribution at $z=0.1$, notwithstanding the substantial WHIM in the IGM. To investigate the effects of different feedback, we apply this inference methodology to mock spectra derived from the IllustrisTNG and Illustris simulations at $z=0.1$. The results suggest that the underlying $[T_0,\gamma]$ of both simulations can be recovered with biases as low as $|\Delta \log(T_0/\text{K})| \lesssim 0.05$ dex, $|\Delta \gamma | \lesssim 0.1$, smaller than the precision of a typical measurement. Given the large differences in the volume-weighted WHIM fractions between the three simulations (Illustris 38\%, IllustrisTNG 10\%, Nyx 4\%) we conclude that the $b$-$N_{H I}$ distribution is not sensitive to the WHIM under realistic conditions. Finally, we investigate the physical properties of the detectable Lyman-$\alpha$ absorbers, and discover that although their $T$ and $\Delta$ distributions remain mostly unaffected by feedback, they are correlated with the photoionization rate used in the simulation. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.13598.md
    + _build/html/tmp_2308.13598/./figures/mom_zero_af.png
    + _build/html/tmp_2308.13598/./figures/mom_zero_gl.png
    + _build/html/tmp_2308.13598/./figures/mom_zero_mr.png
    + _build/html/tmp_2308.13598/./figures/IRS7B_ch3oh_mom_zero.png
exported in  _build/html/2308.14590.md
    + _build/html/tmp_2308.14590/./figures/fig6a.png
    + _build/html/tmp_2308.14590/./figures/fig6b.png
    + _build/html/tmp_2308.14590/./figures/fig2a.png
    + _build/html/tmp_2308.14590/./figures/fig2b.png
    + _build/html/tmp_2308.14590/./figures/fig3a.png
    + _build/html/tmp_2308.14590/./figures/fig3b.png
    + _build/html/tmp_2308.14590/./figures/fig3c.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\change}[1]{#1}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\arraystretch}{1.6}$
$\newcommand{\arraystretch}{1.8}$
$\newcommand{\arraystretch}{1.6}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.8}$</div>



<div id="title">

# Linking ice and gas in the Coronet cluster in Corona Australis

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.13598-b31b1b.svg)](https://arxiv.org/abs/2308.13598)<mark>Appeared on: 2023-08-29</mark> -  _20 pages, 17 figures, accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

<mark>G. Perotti</mark>, et al.

</div>
<div id="abstract">

**Abstract:** During the journey from the cloud to the disc, the chemical composition of the protostellar envelope material can be either preserved or processed to varying degrees depending on the surrounding physical environment. This works aims to constrain the interplay of solid (ice) and gaseous methanol ($CH_3$ OH) in the outer regions of protostellar envelopes located in the Coronet cluster in Corona Australis (CrA), and assess the importance of irradiation by the Herbig Ae/Be star R CrA. $CH_3$ OH is a prime test-case as it predominantly forms as a consequence of the solid-gas interplay (hydrogenation of condensed CO molecules onto the grain surfaces) and it  plays an important role in future complex molecular processing. We present 1.3 mm Submillimeter Array (SMA) and Atacama Pathfinder Experiment (APEX) observations towards the envelopes of four low-mass protostars in the Coronet. Eighteen molecular transitions of seven species are identified. We calculate $CH_3$ OH gas-to-ice ratios in this strongly irradiated cluster and compare them with ratios determined towards protostars located in less irradiated regions such as the Serpens SVS 4 cluster in Serpens Main and the Barnard 35A cloud in the $\lambda$ Orionis region. The $CH_3$ OH gas-to-ice ratios in the Coronet vary by one order of magnitude (from 1.2 $\times$ 10 $^{-4}$ to 3.1 $\times$ 10 $^{-3}$ ) which is similar to less irradiated regions as found in previous studies. We find that the $CH_3$ OH gas-to-ice ratios estimated in these three regions are remarkably similar despite the different UV radiation field intensities and formation histories. This result suggests that the overall $CH_3$ OH chemistry in the outer regions of low-mass envelopes is relatively independent of variations in the physical conditions and hence that it is set during the prestellar stage.

</div>

<div id="div_fig1">

<img src="tmp_2308.13598/./figures/mom_zero_af.png" alt="Fig9" width="100%"/>

**Figure 9. -** Primary beam corrected integrated intensity maps in Jy beam$^{-1}$km s$^{-1}$ for the different species detected with the SMA. In each plot contours start at 5$\sigma$ and continue in intervals of 5$\sigma$, except for panel a-c for which contours start at 10$\sigma$ and continue in intervals of 10$\sigma$. Shown are (a) CO $J=2-1$($\sigma$= 0.96 Jy beam$^{-1}$km s$^{-1}$), (b) $^{13}$CO $J=2-1$($\sigma$= 0.32 Jy beam$^{-1}$km s$^{-1}$), (b) C$^{18}$O $J=2-1$($\sigma$= 0.13 Jy beam$^{-1}$km s$^{-1}$),  (d) $H_2$CO 3$_{03}-$2$_{02}$($\sigma$= 0.23 mJy beam$^{-1}$km s$^{-1}$), (e) $H_2$CO 3$_{21}-$2$_{20}$($\sigma$= 88 mJy beam$^{-1}$km s$^{-1}$), (f) $H_2$CO 3$_{22}-$2$_{21}$($\sigma$= 0.14 mJy beam$^{-1}$km s$^{-1}$). The circular field-of-view corresponds to the SMA primary beam. The size of the synthesised beam is shown in the lower left corner of each image. The white and orange stars and the orange circle mark the position of the Coronet cluster members as in Fig. \ref{rgb_rcra}. (*mom_zero_part1*)

</div>
<div id="div_fig2">

<img src="tmp_2308.13598/./figures/mom_zero_gl.png" alt="Fig10.1" width="50%"/><img src="tmp_2308.13598/./figures/mom_zero_mr.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Same as Figure \ref{mom_zero_part1}. Primary beam corrected integrated intensity maps in Jy beam$^{-1}$km s$^{-1}$ for: (g) $CH_3$OH $4_2-$$3_1$($\sigma$= 88 mJy beam$^{-1}$km s$^{-1}$), (h) $CH_3$OH 5$_{+0}-$4$_{+0}$ E ($\sigma$= 0.12 Jy beam$^{-1}$km s$^{-1}$), (i) $CH_3$OH 5$_{-1}-$4$_{-1}$ E ($\sigma$=0.13 Jy beam$^{-1}$km s$^{-1}$), (j) $CH_3$OH 5$_{0}-$4$_{0}$ A$^{+}$($\sigma$= 0.17 Jy beam$^{-1}$km s$^{-1}$), (k) $CH_3$OH 5$_{+1}-$4$_{+1}$ E ($\sigma$= 0.13 Jy beam$^{-1}$km s$^{-1}$), and (l) $CH_3$OH 5$_{-2}-$4$_{-2}$ E ($\sigma$= 0.15 mJy beam$^{-1}$km s$^{-1}$) detected by the SMA (panel g), and in the combined SMA and APEX data (panels h-l).Same as Figure \ref{mom_zero_part1}. Primary beam corrected integrated intensity maps in Jy beam$^{-1}$km s$^{-1}$ for : (m) SO $5_5-4_4$($\sigma$= 0.12 Jy beam$^{-1}$km s$^{-1}$), (n) SO $6_5-5_4$($\sigma$= 0.13 Jy beam$^{-1}$km s$^{-1}$), (o) $SO_2$ 4$_{22}-3_{13}$($\sigma$= 0.15 Jy beam$^{-1}$km s$^{-1}$), (p) $c$-$C_3$$H_2$ 6$_{06}-$5$_{15}$($\sigma$= 0.13 Jy beam$^{-1}$km s$^{-1}$), (q) $c$-$C_3$$H_2$ 5$_{14}-$4$_{23}$($\sigma$= 0.11 Jy beam$^{-1}$km s$^{-1}$), and (r) $HC_3$N $J=24-23$($\sigma$= 71 mJy beam$^{-1}$km s$^{-1}$) detected by the SMA. (*mom_zero_part2*)

</div>
<div id="div_fig3">

<img src="tmp_2308.13598/./figures/IRS7B_ch3oh_mom_zero.png" alt="Fig7" width="100%"/>

**Figure 7. -** Primary beam corrected integrated intensity maps for $CH_3$OH $J=5_0 - 4_0$ A$^+$ transition ($E_\mathrm{u}=34.8 $K) at 241.791 GHz detected by the SMA (a), by the APEX telescope (b), and in the combined interferometric SMA and single-dish APEX data (c). All lines are integrated between 6 and 12.5 km s$^{-1}$. Contours start at 5$\sigma$($\sigma_\mathrm{SMA}$= 0.16 Jy beam$^{-1}$km s$^{-1}$, $\sigma_\mathrm{APEX}$= 2 Jy beam$^{-1}$km s$^{-1}$, $\sigma_\mathrm{SMA+APEX}$= 0.17 Jy beam$^{-1}$km s$^{-1}$) and follow in steps of 5$\sigma$. The circular field-of-view corresponds to the primary beam of the SMA observations. The synthesised beams are displayed in white in the bottom left corner of each panel. (*mom0_meth*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.13598"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feng}[1]$
$\newcommand{\paola}[1]$
$\newcommand{\uat}[2]{\href{http://vocabs.ands.org.au/repository/api/lda/aas/the-unified-astronomy-thesaurus/current/resource.html?uri=http://astrothesaurus.org/uat/#1}{#2 (#1)}}$
$\newcommand{\Msol}{\hbox{M_{\odot}}}$
$\newcommand{\Lya}{\hbox{Ly\alpha}}$
$\newcommand{\Lyb}{\hbox{Ly\beta}}$
$\newcommand{\Jup}{\hbox{J^\prime}}$
$\newcommand{\Jlo}{\hbox{J^{\prime\prime}}}$
$\newcommand{\Vup}{\hbox{v^\prime}}$
$\newcommand{\Vlo}{\hbox{v^{\prime\prime}}}$
$\newcommand{\Elo}{\hbox{E^{\prime\prime}}}$
$\newcommand{\kms}{\hbox{km s^{-1}}}$
$\newcommand{\erg}{\hbox{erg cm^{-2} s^{-1}}}$
$\newcommand{\erga}{\hbox{erg cm^{-2} s^{-1} Å^{-1}}}$
$\newcommand{\mdotyr}{\hbox{M_\odot yr^{-1}}}$
$\newcommand{\mdot}{\hbox{\dot{M}}}$
$\newcommand{\IUE}{\textit{IUE}}$
$\newcommand{\Chandra}{\textit{Chandra}}$
$\newcommand{\HST}{\textit{HST}}$
$\newcommand{\STIS}{STIS}$
$\newcommand{\FUSE}{\textit{FUSE}}$
$\newcommand{\IDL}{IDL}$
$\newcommand{\vsini}{\hbox{v \sin i}}$
$\newcommand{\ycmod}[1]{\textcolor{olive}{ #1}}$</div>



<div id="title">

# Twenty-Five Years of Accretion onto the Classical T Tauri Star TW Hya

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.14590-b31b1b.svg)](https://arxiv.org/abs/2308.14590)<mark>Appeared on: 2023-08-29</mark> -  _Accepted by ApJ. 31 pages_

</div>
<div id="authors">

G. J. H. (沈雷歌), et al. -- incl., <mark>R. Launhardt</mark>

</div>
<div id="abstract">

**Abstract:** Accretion plays a central role in the physics that governs the evolution and dispersal of protoplanetary disks.  The primary goal of this paper is to analyze the stability over time of the mass accretion rate onto TW Hya, the nearest accreting solar-mass young star.  We measure veiling across the optical spectrum in 1169 archival high-resolution spectra of TW Hya, obtained from 1998--2022.  The veiling is then converted to accretion rate using 26 flux-calibrated spectra that cover the Balmer jump.  The accretion rate measured from the excess continuum has an average of $2.51\times10^{-9}$ M $_\odot$ yr $^{-1}$ and a Gaussian distribution with a FWHM of 0.22 dex.  This accretion rate may be underestimated by a factor of up to 1.5 because of uncertainty in the bolometric correction and another factor of 1.7 because of excluding the fraction of accretion energy that escapes in lines, especially Ly $\alpha$ .The accretion luminosities are well correlated with He line luminosities but poorly correlated with H $\alpha$ and H $\beta$ luminosity.The accretion rate is always flickering over hours but on longer timescales has been stable over 25 years.This level of variability is consistent with previous measurements for most, but not all, accreting young stars.

</div>

<div id="div_fig1">

<img src="tmp_2308.14590/./figures/fig6a.png" alt="Fig10.1" width="50%"/><img src="tmp_2308.14590/./figures/fig6b.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Veiling spectra for strong (pink circles, average relative veiling $r_{5050}=1.52$), above average (blue diamonds, $r_{5050}=0.95$), below average (red squares, $r_{5050}=0.52$), and weak (black triangles, $r_{5050}=0.21$) veiling epochs measured in ESPaDOnS spectra.  The left panel shows the relative veiling measurement, with features in the veiling spectrum caused by the shape of the photospheric template, TWA 25.  The right panel shows the veiling spectrum multiplied by TWA 25.  The spectra with moderate accretion are flat from 4000--9000 Å, while the strongest accretion spectrum has a bluer slope (increasing flux with decreasing wavelength). (*fig:veilspec*)

</div>
<div id="div_fig2">

<img src="tmp_2308.14590/./figures/fig2a.png" alt="Fig7.1" width="50%"/><img src="tmp_2308.14590/./figures/fig2b.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** _ Left:_ The relative veiling (veiling relative to the low-veiling spectrum of TW Hya) in six wavelength regions (y-axis) compared to the veiling from 5000-5100 Å (x-axis), as measured from ESPaDOnS spectra.  Each veiling measurement plotted here is the average of 4--8 different 25 Å spectral regions around the labeled wavelength.  The line shows a 1:1 relationship and not a fit. The veiling at bluer wavelengths is generally higher than the veiling at red wavelengths, with exceptions. The veiling at 4700 Å is similar to that at 5050 Å because of similar photospheric fluxes.
_ Right:_ Correlated differences between measured and expected veiling at long wavelengths.  The inset shows the correlation between veiling at 7000 Å and 5000-5100 Å. The excess veiling is calculated by fitting a line to the $r_{\lambda}$-$r_{5050}$ relationship (as seen on the left panel) and then subtracting each point from that line.  The main plot shows this excess veiling at 7000 Å versus that at 8700 Å.  When the veiling at 7000 Å is higher than expected, the veiling at 8700 Å is also higher than expected.
The correlation between excess veiling at 7000 and 8700 Å demonstrates that the scatter in the correlations between veiling measurements is real and not due to signal-to-noise or other statistical uncertainties. (*fig:veilcomp*)

</div>
<div id="div_fig3">

<img src="tmp_2308.14590/./figures/fig3a.png" alt="Fig8.1" width="33%"/><img src="tmp_2308.14590/./figures/fig3b.png" alt="Fig8.2" width="33%"/><img src="tmp_2308.14590/./figures/fig3c.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** _ Top Left:_ The relationship between the equivalent width of coadded lines (one set shown here for lines from 5250--5400 Å) versus veiling, established from ESPaDOnS spectra and with residuals in ESPaDOnS and FEROS spectra shown below.  _ Top Right:_  Similar plots as on the left for the flux ratio for a spectral dip around 5205--5215 Å compared with a continuum region.  In both cases, we calculate a best-fit relationship between equivalent width (or flux ratios) and veiling with ESPaDOnS.  We then apply those relationships to FEROS spectra.  The bottom panels on the left and right show the scatter between the veiling calculated from these relationships and the veiling measured from comparing the FEROS spectrum to a low-veiling FEROS template.
 _ Bottom:_  The final comparison of veiling obtained from the combination of line equivalent widths and flux ratios to the veiling from a low-veiling FEROS template.  All FEROS spectra are shifted by $\sim 0.02$ to place them on the same scale as the ESPaDOnS veiling measuremenets.
   The scatter of $0.013+0.045 \times r5050$(with a minimum error of 0.02) is applied as an uncertainty to all spectra where veiling is measured from line equivalent widths and flux ratios. (*fig:eqwtoveil*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.14590"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

255  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
